# Prediction of classes using a trained SVM of a bag of words

## Part 1. Import modules and load the data

In [3]:
import cv2 as cv
import numpy as np
import os
from sklearn.svm import LinearSVC
from sklearn.externals import joblib
from scipy.cluster.vq import *
from imutils import paths

DATA_FILENAME = "svm.pkl"

# Load the classifier, class names, scaler, number of clusters and vocabulary 
clf, classes_names, stdSlr, k, voc = joblib.load(DATA_FILENAME)
print("Data loaded! ("+DATA_FILENAME+")")

Data loaded! (svm.pkl)


## Part 2. Read the testing set

In [4]:
test_path = '../data/upf/test_set/'
image_paths = []
testing_categories = os.listdir(test_path)
for test_category in testing_categories:
    dir = os.path.join(test_path, test_category)
    class_path = list(paths.list_images(dir)) # This is the path to all the images given A SINGLE CLASS
    image_paths+=class_path

## Part 3. Calculate keypoints and descriptors

In [5]:
sift = cv.xfeatures2d.SIFT_create()

# List where all the descriptors are stored
des_list = []

for image_path in image_paths:
    im = cv.imread(image_path)
    if im.all == None:
        print("No such file {}\nCheck if the file exists".format(image_path))
    kpts, des = sift.detectAndCompute(im, None)
    des_list.append((image_path, kpts, des))

### Part 4. Stack the descriptors in a vertical numpy array

In [6]:
# Stack all the descriptors vertically in a numpy array
descriptors = des_list[0][2]
for image_path, keypoint, descriptor in des_list[0:]:
    descriptors = np.vstack((descriptors, descriptor))

In [7]:
# Calculate the histogram for the new images
test_features = np.zeros((len(image_paths), k), "float32")
for i in range(len(image_paths)):
    words, distance = vq(des_list[i][2],voc)
    for w in words:
        test_features[i][w] += 1

In [8]:
# Perform Tf-Idf vectorization
# Number of total occurences (per word)
nbr_occurences = np.sum(test_features > 0, axis = 0)
# Inverse Document Frequency
idf = np.array(np.log((1.0*len(image_paths)+1) / (1.0*nbr_occurences + 1)), 'float32')

# Scale the features
test_features = stdSlr.transform(test_features)

In [10]:
# Perform the predictions
predictions =  [classes_names[i] for i in clf.predict(test_features)]

['anchor', 'anchor', 'anchor', 'anchor', 'bass', 'bass', 'bass', 'bass', 'ceiling_fan', 'ceiling_fan', 'ceiling_fan', 'ceiling_fan']


In [54]:
for image_path, prediction in zip(image_paths, predictions):
    image = cv.imread(image_path)
    cv.namedWindow("Image", cv.WINDOW_NORMAL)
    pt = (0, 3 * image.shape[0] // 4)
    cv.putText(image, prediction, pt ,cv.FONT_HERSHEY_SIMPLEX, 2, [0, 255, 0], 2)
    cv.imshow("Image", image)
    cv.waitKey(1000)
    

In [55]:
cv.destroyAllWindows()